In [23]:
from azureml.core import Workspace, Experiment
from azureml.core.compute_target import ComputeTargetException
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: udacityworkspace
Azure region: canadacentral
Subscription id: e9204f07-b8a0-4dfa-b8ca-153a9a4abda8
Resource group: udacity_course


In [24]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "udacity-project"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [25]:
from azureml.core import ScriptRunConfig, Environment
env = Environment.get(workspace=ws, name="AzureML-Tutorial")

In [26]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, quniform
from azureml.train.estimator import Estimator
from azureml.train.hyperdrive import choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C":choice(1,3),
    "--max_iter":choice(100,1000)
})


# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")


est = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config= est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=100,
                             max_concurrent_runs=4)

In [27]:

# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)
hyperdrive_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project1,HD_f7bf8d8c-80e6-4055-8f4b-a430d2106e9c,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [32]:
hyperdrive_run.get_status()

'Completed'

In [29]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [33]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
if best_run is None:
    raise Exception("No best run was found")
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\nAccuracy:', best_run_metrics['Accuracy'])
print('\n')
print(best_run_metrics)
print('\n')
print(best_run)

Best Run Id:  HD_f7bf8d8c-80e6-4055-8f4b-a430d2106e9c_1

Accuracy: 0.9060451565914057


{'Regularization Strength:': 1.0, 'Max iterations:': 100, 'Accuracy': 0.9060451565914057}


Run(Experiment: udacity-project1,
Id: HD_f7bf8d8c-80e6-4055-8f4b-a430d2106e9c_1,
Type: azureml.scriptrun,
Status: Completed)


In [34]:
best_run.get_file_names()

['logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/rslex.log',
 'outputs/model.pkl',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [35]:
model = best_run.register_model(model_name='hyper_model', model_path=os.path.join('outputs', 'model.pkl'))
print(model.name, model.version, sep = '\t')

hyper_model	1


In [36]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

dataset_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
data = TabularDatasetFactory.from_delimited_files(path=dataset_path)

In [37]:
from train import clean_data
from sklearn.model_selection import train_test_split
# Use the clean_data function to clean your data.
x, y = clean_data(data)
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=42,test_size=0.25,stratify=y)

In [38]:
import pandas as pd
train_data = pd.concat([x_train,y_train],axis =1)
train_data.to_csv('./training/train_data.csv')

In [39]:
from azureml.core.dataset import Dataset
ds_tr = ws.get_default_datastore()
ds_tr.upload(src_dir='./training', target_path='training_data', overwrite=True, show_progress=True)
training_data =TabularDatasetFactory.from_delimited_files(path=ds_tr.path('training_data/train_data.csv'))

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


In [40]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric="accuracy",
    compute_target = cpu_cluster,
    training_data=training_data,
    label_column_name='y',
    n_cross_validations=5)

In [41]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output=True)


Submitting remote run.
No run_configuration provided, running on udacity-project with default configuration
Running on remote compute: udacity-project


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project1,AutoML_b96a5fa2-55c8-4a08-b907-ed363bb8e56e,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Nu

In [42]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [63]:
# Retrieve and save your best automl model.
from azureml.automl.runtime.onnx_convert import OnnxConverter
automl_best_run, fitted_model = automl_run.get_output()
model_AutoML = OnnxConverter.save_onnx_model(fitted_model, file_path="./automl_model.onnx")

<function azureml.core.run.Run.list(experiment, type=None, tags=None, properties=None, status=None, include_children=False, _rehydrate_runs=True)>